## Import and load keys

In [24]:
import os
import requests
from dotenv import load_dotenv
from mistralai import Mistral
import json
import gradio as gr
import random

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave de API de Mistral desde las variables de entorno
api_key = os.getenv('MISTRAL_API_KEY')
client = Mistral(api_key=api_key)
model = "mistral-small-latest"

## World Creation

In [3]:
system_prompt = f"""
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.
"""

world_prompt = f"""
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:"""


output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_prompt}
    ],
    temperature=0.9
)

In [4]:
world_output =output.choices[0].message.content
print(world_output)
world_output = world_output.strip()
world = {
    "name": world_output.split('\n')[0].strip()
    .replace('World Name: ', ''),
    "description": '\n'.join(world_output.split('\n')[1:])
    .replace('World Description:', '').strip()
}

World Name: Colossus Archipelago

World Description: In the vast ocean of Aeterna, cities are not built on land but on the backs of colossal, sentient beasts called Terrastrides. These gentle giants roam the seas, their massive shells providing a stable platform for human habitation. Each city is unique, reflecting the personality and preferences of its Terrastride, with architecture that blends organic growth and human ingenuity. The people of the Colossus Archipelago live in harmony with their beastly hosts, forming a symbiotic society that traverses the endless ocean.


## Kingdom Creation

In [5]:
kingdom_prompt = f"""
Create 3 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, history of the kingdom.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1"""


output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": kingdom_prompt}
    ],
)

In [6]:
kingdoms = {}
kingdoms_output = output.choices[0].message.content

for output in kingdoms_output.split('\n\n'):
  kingdom_name = output.strip().split('\n')[0] \
    .split('Name: ')[1].strip()
  print(f'Created kingdom "{kingdom_name}" in {world["name"]}')
  kingdom_description = output.strip().split('\n')[1] \
    .split('Description: ')[1].strip()
  kingdom = {
      "name": kingdom_name,
      "description": kingdom_description,
      "world": world['name']
  }
  kingdoms[kingdom_name] = kingdom
world['kingdoms'] = kingdoms

print(f'\nKingdom 1 Description: \
{kingdom["description"]}')

Created kingdom "Aetheria" in Colossus Archipelago
Created kingdom "Verdantia" in Colossus Archipelago
Created kingdom "Ignisia" in Colossus Archipelago

Kingdom 1 Description: Ignisia is a fiery kingdom that rides the waves atop the molten Terrastride, Pyro. King Ember, a charismatic leader with a fiery spirit, governs his people with a strong sense of honor and tradition. Ignisians are renowned for their blacksmithing and glassblowing skills, their culture centered around forges and foundries. The kingdom's history is one of constant struggle against the harsh conditions of their Terrastride, forging a people of unyielding strength and determination.


## Town Creation

In [7]:
def get_town_prompt(world, kingdom):
    return f"""
    Create 3 different towns for a fantasy kingdom and world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \
    
    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}
    
    Town 1 Name:"""
    
import time

import time  # Asegúrate de importar la biblioteca time

def create_towns(world, kingdom):
    print(f'\nCreating towns for kingdom: {kingdom["name"]}...')
    output = client.chat.complete(
      model=model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_town_prompt(world, kingdom)}
      ],
      temperature=0.9,
  )
    towns_output = output.choices[0].message.content
    
    towns = {}
    for output in towns_output.split('\nCreating towns for kingdom: {kingdom["name"]}...'):
        for num in range(1, 4):
            print(output.split(f"Town {num} Name: ")[1].split("\n")[0])
            print(output.split(f"Town {num} Description: ")[1].split("\n")[0])
            print("\n")
         
            town_name = output.split(f"Town {num} Name: ")[1].split("\n")[0] 
            town_description = output.split(f"Town {num} Description: ")[1].split("\n")[0]
        
            town = {
              "name": town_name,
              "description": town_description,
              "world": world['name'],
              "kingdom": kingdom['name']
            }
            towns[town_name] = town
        kingdom["towns"] = towns

In [8]:
for kingdom in kingdoms.values():
    create_towns(world, kingdom)  

first_kingdom = list(kingdoms.values())[0]
print(first_kingdom)
town = list(first_kingdom['towns'].values())[0]
print(f'\nTown 1 Description: {town["description"]}')


Creating towns for kingdom: Aetheria...
Lumina
Nestled in the heart of the kingdom, Lumina is a beacon of learning and innovation. The town's centerpiece is the Grand Library, a sprawling structure filled with scrolls and tomes from across the archipelago. The town's history is marked by the invention of the Luminary, a device that harnesses the power of bioluminescent plants to provide light, revolutionizing life on the Terrastrides. The town is also home to the Academy of Arcane Arts, where scholars study the magical properties of the ocean and its inhabitants.


Ventus
Perched on the edge of Sapientia's shell, Ventus is a bustling port town, known for its windmills and sails that harness the power of the wind. The town's history is tied to the great schism, as it was here that the dissenting faction first attempted to harness Terrastride power for personal gain. Now, Ventus is a symbol of redemption, with its Wind Council dedicated to ethical wind energy use. The town's market is a

## Npcs Creation

In [9]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \
    
    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}
    
    Town Name: {town['name']}
    Town Description: {town['description']}
    
    Character 1 Name:"""
    

In [10]:
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.chat.complete(
      model= model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_npc_prompt(world, kingdom, town)}
      ],
        temperature=0.7 
    )

    npcs_output = output.choices[0].message.content
    npcs = {}
    for num in range(1, 4):
        print(npcs_output.split(f"Character {num} Name: ")[1].split("\n")[0])
        print(npcs_output.split(f"Character {num} Description: ")[1].split("\n")[0])
        print("\n")

        character_name = npcs_output.split(f"Character {num} Name: ")[1].split("\n")[0]
        character_description = npcs_output.split(f"Character {num} Description: ")[1].split("\n")[0]

        character = {
        "name": character_name,
        "description": character_description,
        "world": world['name'],
        "kingdom": kingdom['name'],
        "town": town['name']
        }
        npcs[character_name] = character
        town["npcs"] = npcs

    
for kingdom in kingdoms.values():
    for town in kingdom['towns'].values():
        create_npcs(world, kingdom, town)
  # For now we'll only generate npcs for one kingdom and town
    break


Creating characters for the town of: Lumina...
Elara Vesperia
Elara is a tall, slender woman in her early thirties, with sun-kissed skin and eyes that sparkle with curiosity. Her hair, a cascade of dark curls, is often adorned with small, intricate braids that hold tiny vials of glowing luminescent plants. As a renowned scholar at the Academy of Arcane Arts, she spends her days studying the magical properties of the ocean, but her true desire is to uncover the secrets of the Terrastrides' ancient magic, a pursuit driven by the mysterious illness that plagues her younger sister, who lives in a distant city, and whom she hopes to cure.


Orion Thorne
Orion is a burly man in his mid-forties, his muscular frame a testament to years of labor in the kingdom's workshops. His face is weathered, with deep lines etched by the sun and salt spray, and his hands are calloused from years of work. As the chief engineer of Aetheria, he oversees the creation of innovative technologies, but his heart a

In [26]:
npc = list(town['npcs'].values())[0]

print(f'\nNPC 1 in {town["name"]}, \
{kingdom["name"]}:\n{npc["description"]}')

TypeError: string indices must be integers, not 'str'

In [12]:
world["name"]

'Colossus Archipelago'

## Save World

In [13]:
def save_world(world, filename):
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as f:
        json.dump(world, f)

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f) 

save_world(world, f'../shared_data/{world["name"]}.json')

## Generating Text box

In [18]:
demo = None
def start_game(main_loop, share=False):
    # added code to support restart
    global demo
    # If demo is already running, close it first
    if demo is not None:
        demo.close()

    demo = gr.ChatInterface(
        main_loop,
        chatbot=gr.Chatbot(height=250, placeholder="Type 'start game' to begin"),
        textbox=gr.Textbox(placeholder="What do you do next?", container=False, scale=7),
        title="AI RPG",
        # description="Ask Yes Man any question",
        theme="soft",
        examples=["Look around", "Continue the story"],
        cache_examples=False,
        #retry_btn="Retry",
        #undo_btn="Undo",
        #clear_btn="Clear",
                           )
    demo.launch(share=share, server_name="0.0.0.0")

def test_main_loop(message, history):
    return 'Entered Action: ' + message

# start_game(test_main_loop)

## Generating Start

In [47]:
world_name = world["name"]
world = load_world(f'../shared_data/{world_name}.json')
# Access the world, kingdom, town and character. Here is the first of each selected.
#kingdom = random.choice(list(world['kingdoms'].keys()))
kingdom = list(world['kingdoms'].keys())[0] 
town = random.choice(list(world['kingdoms'][kingdom]['towns'].keys()))
character = random.choice(list(world['kingdoms'][kingdom]['towns'][town]['npcs'].keys()))

In [29]:
system_prompt = """You are an AI Game master. Your job is to create a 
start to an adventure based on the world, kingdom, town and character 
a player is playing as. 
Instructions:
You must only use 2-4 sentences \
Write in second person. For example: "You are Jack" \
Write in present tense. For example "You stand at..." \
First describe the character and their backstory. \
Then describes where they start and what they see around them."""
world_info = f"""
World: {world}
Kingdom: {kingdom}
Town: {town}
Your Character: {character}
"""

model_output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info + '\nYour Start:'}
    ],
    temperature=1.0
)
start = model_output.choices[0].message.content
print(start)
world['start'] = start
#save_world(world, f'../shared_data/{world_name}.json')  
save_world(world, f'../shared_data/{world_name}.json')

You are Lyra Aetheris, the Archon of Aetheria, known for your sharp mind and sharper tongue. Your cybernetic eye glows softly, a remnant of an experiment gone wrong, and your average height belies the immense responsibility you carry. You stand at the edge of the sprawling deck of Ventus, the wind whipping around you as the windmills hum in the background. The vast ocean stretches out before you, the sun glinting off the waves, and the scent of saltwater fills the air. Below, the gentle giant Terrastride, Sapientia, moves slowly, its massive shell supporting the bustling town. The market below is a vibrant mix of cultures, with traders from across the archipelago peddling their wares.


## Creating the Main Action Loop

In [30]:
def run_action(message, history, game_state):
    
    if(message == 'start game'):
        return game_state['start']

    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
You must on only write 1-3 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...)"""
    
    world_info = f"""
World: {game_state['world']}
Kingdom: {game_state['kingdom']}
Town: {game_state['town']}
Your Character:  {game_state['character']}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info}
    ]
    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})

    messages.append({"role": "user", "content": message})
    model_output = client.chat.complete(
        model=model,
        messages=messages
    )
    
    result = model_output.choices[0].message.content
    return result

In [48]:
game_state = {
    "world": world['description'],
    "kingdom": world["kingdoms"][kingdom]['description'],
    "town": world["kingdoms"][kingdom]["towns"][town]['description'],
    "character": world['kingdoms'][kingdom]['towns'][town]['npcs'][character]['description'],
    "start": start,
}

def main_loop(message, history):
    return run_action(message, history, game_state)

In [55]:
game_state

{'world': 'In the vast ocean of Aeterna, cities are not built on land but on the backs of colossal, sentient beasts called Terrastrides. These gentle giants roam the seas, their massive shells providing a stable platform for human habitation. Each city is unique, reflecting the personality and preferences of its Terrastride, with architecture that blends organic growth and human ingenuity. The people of the Colossus Archipelago live in harmony with their beastly hosts, forming a symbiotic society that traverses the endless ocean.',
 'kingdom': "Aetheria is a kingdom of scholars and inventors, perched atop the shell of the wise and slow-moving Terrastride, Sapientia. The kingdom is ruled by Archon Lyra, a brilliant engineer who oversees the development of innovative technologies that harness the power of wind and water. Aetherians value knowledge and progress, their culture revolving around grand libraries, bustling laboratories, and lively debates. The kingdom's history is marked by a 

In [56]:
def get_game_state():
    world = load_world(f'../shared_data/{world_name}.json')
    kingdom = world["kingdoms"][kingdom]
    town = world["kingdoms"][kingdom]["towns"][town]
    character = world['kingdoms'][kingdom]['towns'][town]['npcs'][character]
    start = world['start']

    game_state = {
        "world": world['description'],
        "kingdom": world["kingdoms"][kingdom]['description'],
        "town": world["kingdoms"][kingdom]["towns"][town]['description'],
        "character": world['kingdoms'][kingdom]['towns'][town]['npcs'][character]['description'],
        "start": start,
    }
    return game_state

## Launch and Share!

In [ ]:
# start_game(main_loop, True)

In [ ]:
# from helper import get_game_state

# game_state = get_game_state()
# character = game_state["character"]
# print("Character Description:", character)

## Define Inventory Detector

In [50]:
system_prompt = """You are an AI Game Assistant. \
Your job is to detect changes to a player's \
inventory based on the most recent story and game state.
If a player picks up, or gains an item add it to the inventory \
with a positive change_amount.
If a player loses an item remove it from their inventory \
with a negative change_amount.
Given a player name, inventory and story, return a list of json update
of the player's inventory in the following form.
Only take items that it's clear the player (you) lost.
Only give items that it's clear the player gained. 
Don't make any other item updates.
If no items were changed return {"itemUpdates": []}
and nothing else.

Response must be in Valid JSON
Don't add items that were already added in the inventory

Inventory Updates:
{
    "itemUpdates": [
        {"name": <ITEM NAME>, 
        "change_amount": <CHANGE AMOUNT>}...
    ]
}
"""

In [51]:
def detect_inventory_changes(game_state, output):
    
    inventory = game_state['inventory']
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": 
         f'Current Inventory: {str(inventory)}'},
        
        {"role": "user", "content": f'Recent Story: {output}'},
        {"role": "user", "content": 'Inventory Updates'}
    ]
    chat_completion = client.chat.complete(
        # response_format={"type": "json_object", "schema": InventoryUpdate.model_json_schema()},
        model=model,
        temperature=0.0,
        messages=messages
    )
    response = chat_completion.choices[0].message.content
    result = json.loads(response)
    return result['itemUpdates']

In [54]:
print(kingdom)
print(world["kingdoms"][kingdom]
)

Aetheria
{'name': 'Aetheria', 'description': "Aetheria is a kingdom of scholars and inventors, perched atop the shell of the wise and slow-moving Terrastride, Sapientia. The kingdom is ruled by Archon Lyra, a brilliant engineer who oversees the development of innovative technologies that harness the power of wind and water. Aetherians value knowledge and progress, their culture revolving around grand libraries, bustling laboratories, and lively debates. The kingdom's history is marked by a great schism, when a faction sought to exploit Terrastrides for personal gain, leading to a strict code of harmony with their beastly hosts.", 'world': 'Colossus Archipelago', 'towns': {'Lumina': {'name': 'Lumina', 'description': "Nestled in the heart of the kingdom, Lumina is a beacon of learning and innovation. The town's centerpiece is the Grand Library, a sprawling structure filled with scrolls and tomes from across the archipelago. The town's history is marked by the invention of the Luminary, a

In [52]:
game_state = get_game_state()
game_state['inventory'] = {
    "cloth pants": 1,
    "cloth shirt": 1,
    "gold": 5
}

result = detect_inventory_changes(game_state, 
"You buy a sword from the merchant for 5 gold")

print(result)

UnboundLocalError: cannot access local variable 'kingdom' where it is not associated with a value

In [ ]:
def update_inventory(inventory, item_updates):
    update_msg = ''
    
    for update in item_updates:
        name = update['name']
        change_amount = update['change_amount']
        
        if change_amount > 0:
            if name not in inventory:
                inventory[name] = change_amount
            else:
                inventory[name] += change_amount
            update_msg += f'\nInventory: {name} +{change_amount}'
        elif name in inventory and change_amount < 0:
            inventory[name] += change_amount
            update_msg += f'\nInventory: {name} {change_amount}'
            
        if name in inventory and inventory[name] < 0:
            del inventory[name]
            
    return update_msg


## Story with inventory

In [ ]:
def run_action(message, history, game_state):
    
    if(message == 'start game'):
        return game_state['start']
        
    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
You must on only write 1-3 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...) \
Don't let the player use items they don't have in their inventory.
"""

    world_info = f"""
World: {game_state['world']}
Kingdom: {game_state['kingdom']}
Town: {game_state['town']}
Your Character:  {game_state['character']}
Inventory: {json.dumps(game_state['inventory'])}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info}
    ]

    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})
           
    messages.append({"role": "user", "content": message})
    model_output = client.chat.complete(
        model=model,
        messages=messages
    )
    
    result = model_output.choices[0].message.content
    return result

In [ ]:
game_state = get_game_state(inventory={
    "cloth pants": 1,
    "cloth shirt": 1,
    "goggles": 1,
    "leather bound journal": 1,
    "gold": 5
})

In [ ]:

def main_loop(message, history):
    output = run_action(message, history, game_state)
    
    safe = is_safe(output)
    if not safe:
        return 'Invalid Output'

    item_updates = detect_inventory_changes(game_state, output)
    update_msg = update_inventory(
        game_state['inventory'], 
        item_updates
    )
    output += update_msg

    return output

start_game(main_loop, True)